In [15]:
using LinearAlgebra

In [ ]:
function tsls(d, y, z, x::Union{Nothing, DataFrame, Array} = nothing, intercept::Bool = true, homoscedastic::Bool = true)
    n = size(y, 1)
    d = Matrix(d)
    if !isnothing(x)
        x = Matrix(x)
    end
    z = Matrix(z)
    
    if intercept & isnothing(x)
        x = ones(n, 1)
    elseif intercept
        x = hcat(ones(n, 1), x)
    end
    
    a1 = size(d, 2)
    if isnothing(x)
        a2 = 0
    else
        a2 = size(x, 2)
    end
    
    k = a1 + a2
    
    if isnothing(x)
        X = d
        Z = z
    else
        X = hcat(d, x)
        Z = hcat(z, x)
    end
        
    Mxz = X' * Z
    Mzz = pinv(Z' * Z)
    
    M = pinv(Mxz * Mzz * Mxz')
    
    b = M * Mxz * Mzz * (Z' * y)
    
    if homoscedastic
        e = y - X * b
        VC1 = (e' * e / (n - k)) .* M
        
    elseif !homoscedastic
        e = y - X * b
        S = 0
        for i in 1:n
            S = S + e[i] ^ 2 * (Z[i, :] * Z[i, :]')
        end
        S = S / n
        VC1 = n .* M * (Mxz * Mzz .* S * Mzz * Mxz') * M
    end
    
    res = Dict("coefficients" => b, "vcov" => VC1, "se" => diag(VC1), "residuals" => e, "sample_size" => n)
    return res
end
    